External lib <br>
- pip install spotipy
- pip install pandas

In [6]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json

In [13]:
client_credentials_manager = SpotifyClientCredentials(client_id="41cec0a047534032841497667966932b", client_secret="5f5f1fa8864c4026add52b90006bddbd")

In [14]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [20]:
playlist_link='https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF'

In [22]:
playlist_URI=playlist_link.split("/")[-1]

In [23]:
data = sp.playlist_tracks(playlist_URI)

In [24]:
# Convert data to JSON format
json_data = json.dumps(data)

# Save JSON data to a file
with open('data.json', 'w') as f:
    f.write(json_data)